In [1]:
import numpy as np
from scipy.constants import c, epsilon_0, mu_0, hbar, Boltzmann, e, physical_constants
from pint import UnitRegistry, set_application_registry
ureg = UnitRegistry()
set_application_registry(ureg)
from arc import Rubidium85

from cavities import find_suitable_transitions, get_transition_parameters, cavity_dims_from_aspect_ratios, Cavity

atom = Rubidium85()
c = c * ureg.meter / ureg.second
mu_0 = mu_0 * ureg.newton / ureg.ampere**2
epsilon_0 = epsilon_0 * ureg.farad / ureg.meter
hbar = hbar * ureg.joule * ureg.second
bohr_radius = physical_constants['Bohr radius'][0] * ureg.meter 
e = e * ureg.coulomb


In [6]:
m, n, p = 1, 0, 1

suitable_transitions = [{"n1": t[0], "n2": t[1], "frequency": t[2]} for t in find_suitable_transitions([10 * ureg.gigahertz, 15 * ureg.gigahertz])]
for transition in suitable_transitions:
    dipole_matrix_element, Gamma_p = get_transition_parameters(transition["n1"], transition["n2"])
    transition["dipole_matrix_element"] = dipole_matrix_element.to(ureg.debye)
    transition["Gamma_p"] = Gamma_p.to(ureg.kilohertz)
    print(f"Transition {transition["n1"]}D -> {transition["n2"]}F at {transition["frequency"]:#~P} with dipole matrix element {transition["dipole_matrix_element"]:#~P} and Gamma_p {transition["Gamma_p"]:#~P}")

    a, b, d = cavity_dims_from_aspect_ratios(transition["frequency"], m, n, p)
    cavity = Cavity(a, b, d, transition["frequency"], (m, n, p), 1e5, mu=transition["dipole_matrix_element"])

    g_0_max, g_0_avg, V_eff_max, V_eff_avg, E_zpf_max, E_zpf_avg = cavity.cavity_statistics()
    transition["|g_0_max|"] = g_0_max.to(ureg.kilohertz)
    transition["g_0_avg"] = g_0_avg.to(ureg.kilohertz)
    # transition["|V_eff_max|"] = V_eff_max.to(ureg.millimeter**3)
    # transition["V_eff_avg"] = V_eff_avg.to(ureg.millimeter**3)
    transition["|E_zpf_max|"] = E_zpf_max.to(ureg.millivolt / ureg.centimeter)
    transition["E_zpf_avg"] = E_zpf_avg.to(ureg.millivolt / ureg.centimeter)

    driving_power = atom.getDrivingPower(
        n1 = 5, # --> 5P
        l1 = 1, # 5P <--
        j1 = 1.5, # 1 unit of orbital and 0.5 of spin
        mj1 = 1.5, # degenerate? 
        n2 = transition['n1'], # the lower Rydberg state
        l2 = 2, # we just assumed this
        j2 = 2.5,
        mj2 = 2.5,
        q = 1, # right-hand circular polarization, also arbitrary
        rabiFrequency = 5e6, # this is the Rabi frequency we want
        laserWaist = 0.0005, # 2mm
    )

    transition["driving_power"] = (driving_power * ureg.watt).to(ureg.milliwatt)
    # print(f"g_0 max: {g_0_max:#~P}, min: {g_0_min:#~P}, avg: {g_0_avg:#~P}")
    # print(f"V_eff max: {V_eff_max:#~P}, min: {V_eff_min:#~P}, avg: {V_eff_avg:#~P}")
    # print(f"E_zpf max: {E_zpf_max:#~P}, min: {E_zpf_min:#~P}, avg: {E_zpf_avg:#~P}")


Transition 54D -> 53F at 14.723740706248611 GHz with dipole matrix element -6.212256317590297 kD and Gamma_p 85.99521891024114 kHz


/Users/muon/Documents/QPG/mwcav-manual/.venv/lib/python3.12/site-packages/pint/facets/plain/quantity.py:1007: RuntimeWarning:

divide by zero encountered in divide



Transition 55D -> 54F at 13.918435177654391 GHz with dipole matrix element -6.448119554294987 kD and Gamma_p 82.02938821348332 kHz
Transition 56D -> 55F at 13.170797704226624 GHz with dipole matrix element -6.688371356017867 kD and Gamma_p 78.32091984458822 kHz
Transition 57D -> 56F at 12.47575773353525 GHz with dipole matrix element -6.9330117254273596 kD and Gamma_p 74.84820511131983 kHz
Transition 58D -> 57F at 11.828770402015671 GHz with dipole matrix element -7.182040665740189 kD and Gamma_p 71.59247768904488 kHz
Transition 59D -> 58F at 11.225754038777184 GHz with dipole matrix element -7.435458179068937 kD and Gamma_p 68.53631508013063 kHz
Transition 60D -> 59F at 10.663036017798007 GHz with dipole matrix element -7.693264268090247 kD and Gamma_p 65.66422400969638 kHz
Transition 61D -> 60F at 10.137305724662866 GHz with dipole matrix element -7.9554589348895774 kD and Gamma_p 62.962092789603645 kHz
Transition 69D -> 67F at 14.45494407944894 GHz with dipole matrix element -6.1900

In [7]:
import plotly.graph_objects as go

# Create table from suitable_transitions
def format_header(key, sample_value):
    """Format header with units in parentheses"""
    if hasattr(sample_value, 'units'):
        return f"{key} ({sample_value.units:~})"
    else:
        return key

def format_value_magnitude(value):
    """Format value showing only magnitude for Pint quantities"""
    if hasattr(value, 'magnitude'):
        return f"{value.magnitude:.3g}"
    else:
        return str(value)

fig = go.Figure(data=[go.Table(
    header=dict(values=[format_header(key, suitable_transitions[0][key]) for key in suitable_transitions[0].keys()],
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[[format_value_magnitude(d[key]) for d in suitable_transitions] for key in suitable_transitions[0].keys()],
               fill_color='lavender',
               align='left'),
    columnwidth=[1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3]
)])


fig.show()
fig.write_html("cavity_exploration.html")
